In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import torch
import pandas as pd
from PIL import ImageDraw, ImageFont, Image
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch
import torch.nn as nn
from transformers import ViTModel
from torchinfo import summary  # 
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
import random
import time
from sklearn.utils.class_weight import compute_class_weight
from transformers import EfficientNetForImageClassification

In [2]:
DEVICE="cuda:0"
def setAllSeeds(seed):
  os.environ['MY_GLOBAL_SEED'] = str(seed)
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
# setAllSeeds(42)

In [3]:
df = pd.read_csv("train.csv")
categories=df["Category"].unique()
print(categories)
category=categories[4]
df = df[df["Category"]==category]

['Men Tshirts' 'Sarees' 'Kurtis' 'Women Tshirts' 'Women Tops & Tunics']


In [4]:
df.isna().sum()

id              0
Category        0
len             0
attr_1       4946
attr_2       5667
attr_3       5938
attr_4       5553
attr_5       6437
attr_6       5787
attr_7       5788
attr_8       5207
attr_9       6433
attr_10     11823
dtype: int64

In [5]:
# attr_i=1
# removing_attri=[]
# df.iloc[:,3:2+attr_i]=np.nan
# df.iloc[:,3+attr_i:]=np.nan


In [6]:
print(df.value_counts())

id     Category             len  attr_1     attr_2   attr_3   attr_4           attr_5  attr_6   attr_7      attr_8         attr_9           attr_10
51376  Women Tops & Tunics  10   navy blue  fitted   crop     high             casual  default  solid       short sleeves  default          knitted    1
64276  Women Tops & Tunics  10   blue       fitted   crop     sweetheart neck  casual  solid    solid       long sleeves   regular sleeves  default    1
64190  Women Tops & Tunics  10   default    boxy     crop     round neck       casual  printed  typography  short sleeves  regular sleeves  tie-ups    1
64193  Women Tops & Tunics  10   maroon     regular  regular  round neck       casual  solid    solid       long sleeves   puff sleeves     ruffles    1
64199  Women Tops & Tunics  10   black      fitted   regular  round neck       casual  solid    solid       long sleeves   regular sleeves  knitted    1
                                                                                       

In [7]:
delCol = []
idxCol = []
trackNum = []
weights=[]
m=nn.Softmax(dim=1)
l=[]
for i in range(1,11):
    uniName = df["attr_"+str(i)].dropna().unique()
    print(uniName)
    if(len(uniName)==0):
        delCol.append("attr_"+str(i))
    else:
        idxCol.append("attr_"+str(i))
        l.append('attr_'+str(i))
        trackNum.append(len(uniName))
        class_weights=compute_class_weight(class_weight="balanced",classes=uniName,y=df["attr_"+str(i)].dropna())
        weights.append(torch.tensor([1]*len(uniName),dtype=torch.float32).to(DEVICE))


['black' 'navy blue' 'red' 'default' 'maroon' 'white' 'green' 'blue'
 'pink' 'yellow' 'peach' 'multicolor']
['regular' 'fitted' 'boxy' 'default']
['crop' 'regular']
['high' 'round neck' 'stylised' 'sweetheart neck' 'v-neck' 'square neck'
 'default']
['casual' 'party']
['default' 'printed' 'solid']
['solid' 'typography' 'graphic' 'default' 'quirky' 'floral']
['short sleeves' 'sleeveless' 'three-quarter sleeves' 'long sleeves']
['regular sleeves' 'default' 'sleeveless' 'puff sleeves']
['knitted' 'default' 'ruffles' 'waist tie-ups' 'tie-ups' 'applique']


In [8]:
df = df.drop(delCol,axis=1)
df.shape

(19004, 13)

In [9]:
df.isna().sum()

id              0
Category        0
len             0
attr_1       4946
attr_2       5667
attr_3       5938
attr_4       5553
attr_5       6437
attr_6       5787
attr_7       5788
attr_8       5207
attr_9       6433
attr_10     11823
dtype: int64

In [10]:
# df.dropna(inplace=True)

In [11]:
id2label={}
label2id={}
attrs={}
total_attr=len(df.columns)
for i in range(3,total_attr):
    labels=df[df.columns[i]].dropna().unique()
    # print(df.columns[i],labels)
    id2label[i-3]={k:labels[k] for k in range(len(labels))}
    label2id[i-3]={labels[k]:k for k in range(len(labels))}
    attrs[i-3]=df.columns[i]
print(id2label)
print(label2id)
print(attrs)

{0: {0: 'black', 1: 'navy blue', 2: 'red', 3: 'default', 4: 'maroon', 5: 'white', 6: 'green', 7: 'blue', 8: 'pink', 9: 'yellow', 10: 'peach', 11: 'multicolor'}, 1: {0: 'regular', 1: 'fitted', 2: 'boxy', 3: 'default'}, 2: {0: 'crop', 1: 'regular'}, 3: {0: 'high', 1: 'round neck', 2: 'stylised', 3: 'sweetheart neck', 4: 'v-neck', 5: 'square neck', 6: 'default'}, 4: {0: 'casual', 1: 'party'}, 5: {0: 'default', 1: 'printed', 2: 'solid'}, 6: {0: 'solid', 1: 'typography', 2: 'graphic', 3: 'default', 4: 'quirky', 5: 'floral'}, 7: {0: 'short sleeves', 1: 'sleeveless', 2: 'three-quarter sleeves', 3: 'long sleeves'}, 8: {0: 'regular sleeves', 1: 'default', 2: 'sleeveless', 3: 'puff sleeves'}, 9: {0: 'knitted', 1: 'default', 2: 'ruffles', 3: 'waist tie-ups', 4: 'tie-ups', 5: 'applique'}}
{0: {'black': 0, 'navy blue': 1, 'red': 2, 'default': 3, 'maroon': 4, 'white': 5, 'green': 6, 'blue': 7, 'pink': 8, 'yellow': 9, 'peach': 10, 'multicolor': 11}, 1: {'regular': 0, 'fitted': 1, 'boxy': 2, 'default'

In [12]:
def categorize(example):
    for i in attrs:
        # print(example[attrs[i]],type(example[attrs[i]]),pd.isna(example[attrs[i]]))
        if not pd.isna(example[attrs[i]]):
            example[attrs[i]]=label2id[i][example[attrs[i]]]
        else:
            example[attrs[i]]=-100
    return example
df=df.apply(categorize,axis=1)
df.head()

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
51209,51375,Women Tops & Tunics,10,0,0,-100,-100,-100,-100,-100,-100,0,-100
51210,51376,Women Tops & Tunics,10,1,1,0,0,0,0,0,0,1,0
51211,51377,Women Tops & Tunics,10,2,0,1,1,0,1,1,1,2,-100
51212,51378,Women Tops & Tunics,10,3,1,0,2,0,2,0,0,0,1
51213,51379,Women Tops & Tunics,10,3,2,1,1,0,1,1,0,1,-100


In [13]:
df.isna().sum()

id          0
Category    0
len         0
attr_1      0
attr_2      0
attr_3      0
attr_4      0
attr_5      0
attr_6      0
attr_7      0
attr_8      0
attr_9      0
attr_10     0
dtype: int64

In [14]:
val_df = df

In [15]:
from sklearn.model_selection import train_test_split


train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
# min_count = train_df[f'attr_{attr_i}'].value_counts().min()
# print(f"The minimum count across all categories is: {min_count}")
# df_sampled = train_df.groupby(f'attr_{attr_i}').sample(n=min_count, random_state=42)
# train_df = df_sampled

In [16]:
for i in l:
    print(train_df[i].value_counts())

attr_1
-100    4465
 0      2429
 5      2193
 3      1765
 8      1366
 7       847
 9       834
 2       723
 4       717
 6       604
 11      409
 1       383
 10      368
Name: count, dtype: int64
attr_2
-100    5115
 1      4949
 0      4353
 2      1835
 3       851
Name: count, dtype: int64
attr_3
 1      6614
-100    5346
 0      5143
Name: count, dtype: int64
attr_4
 1      6247
-100    4993
 0      1662
 5      1077
 4       990
 6       981
 3       712
 2       441
Name: count, dtype: int64
attr_5
 0      11114
-100     5799
 1        190
Name: count, dtype: int64
attr_6
 2      6999
-100    5211
 1      4033
 0       860
Name: count, dtype: int64
attr_7
 0      7227
-100    5216
 3      1551
 1      1402
 4       742
 2       520
 5       445
Name: count, dtype: int64
attr_8
 0      6679
-100    4679
 3      3178
 2      1635
 1       932
Name: count, dtype: int64
attr_9
 0      6724
-100    5787
 3      2466
 1      1471
 2       655
Name: count, dtype: int64
attr_10
-10

In [17]:
# weights[4]=torch.tensor([1,20],dtype=torch.float32).to(DEVICE)
# weights[9]=torch.tensor([1,1,1,1,3,2],dtype=torch.float32).to(DEVICE)
# weights[7]=torch.tensor([1,10],dtype=torch.float32).to(DEVICE)

In [18]:
# for i in l:
#     print(i)
#     for j in set(train_df[i].unique()):
#         x=train_df[i][train_df[i]==j]
#         if x.count()<300 and j!=-100:
#             y=train_df[train_df[i]==j]
#             y.iloc[:,3:]=-100
#             y[i]=j
#             print(y)
#             train_df=pd.concat([train_df,y,y]).reset_index(drop=True)
# for i in l:
#     print(train_df[i].value_counts())

In [19]:
train_df.head()

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
67236,67402,Women Tops & Tunics,10,-100,-100,-100,-100,-100,-100,3,-100,-100,-100
51762,51928,Women Tops & Tunics,10,9,0,1,1,0,1,5,3,0,1
57626,57792,Women Tops & Tunics,10,0,0,-100,-100,-100,-100,-100,-100,-100,0
67281,67447,Women Tops & Tunics,10,4,0,-100,-100,0,-100,-100,2,-100,-100
68305,68471,Women Tops & Tunics,10,7,1,1,1,0,2,0,0,-100,0


In [20]:
# print(train_df[f'attr_{attr_i}'].value_counts())
# print(val_df[f'attr_{attr_i}'].value_counts())

In [21]:
from transformers import AutoImageProcessor
# model_name = 'google/vit-base-patch16-224'
# model_name=f"vit5/{category}/final"
# model_name = 'google/efficientnet-b0'
# model_name = 'vit6/'+category+'/final'
# model_name="google/vit-large-patch16-224"
model_name="Shadowking912/inception_v3"
# model_name="vit6/"+category+"/final"
# processor = AutoImageProcessor.from_pretrained(model_name)

In [22]:
class CustomFashionManager(Dataset):
    def __init__(self,csv_file, root_dir="./",transforms=None):
        self.fashionItems = csv_file
        self.root_dir = root_dir
        self.transforms = transforms
    
    def __len__(self):
        return len(self.fashionItems)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,f"{self.fashionItems.iloc[idx, 0]:06d}"+'.jpg')
        image = Image.open(img_name)
        attributes = self.fashionItems.iloc[idx, 3:]
        attributes = np.array(attributes)
        attributes = attributes.astype('float')
        
        # inputs=processor(image, return_tensors='pt')
        inputs={}
        inputs['labels']=torch.tensor(attributes, dtype=torch.long)
        inputs['pixel_values'] = self.transforms(image).unsqueeze(0)

        return inputs


In [23]:
from torchvision import transforms
transformations=transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
# transformations=None
train_fashion_data = CustomFashionManager(csv_file=train_df,
                                    root_dir='train_images',transforms=transformations)
val_fashion_data = CustomFashionManager(csv_file=val_df,
                                    root_dir='train_images',transforms=transformations)
test_fashion_data = CustomFashionManager(csv_file=df,root_dir='train_images',transforms=transformations)
        

In [24]:
import sys
from typing import List
from transformers import (
    ViTConfig,ViTPreTrainedModel,EfficientNetConfig,EfficientNetPreTrainedModel,
    EfficientNetModel,PretrainedConfig,PreTrainedModel)
from torchvision import models

class CustomConfig(PretrainedConfig):
    def __init__(self,num_classes_per_label:List[int]=[1],**kwargs):
        super().__init__(**kwargs)
        self.num_classes_per_label = num_classes_per_label

            
class MultiLabelMultiClassViT(PreTrainedModel):
    config_class=CustomConfig
    def __init__(self, config: CustomConfig,pretrain=False) -> None:
        super().__init__(config)

        self.model = models.inception_v3(pretrained=pretrain)
        
        self.classifiers = nn.ModuleList([
            # nn.Sequential(nn.Dropout(0.2),
            # nn.Linear(config.hidden_size, 32),
            # nn.ReLU(),
            # nn.Dropout(0.2),
            nn.Linear(self.model.fc.in_features, num_classes)
            for num_classes in config.num_classes_per_label
        ])
        self.classifiers2 = nn.ModuleList([
            # nn.Sequential(nn.Dropout(0.2),
            # nn.Linear(config.hidden_size, 32),
            # nn.ReLU(),
            # nn.Dropout(0.2),
            nn.Linear(self.model.AuxLogits.fc.in_features, num_classes)
            for num_classes in config.num_classes_per_label
        ])
        self.model.fc=nn.Identity()
        self.model.AuxLogits.fc=nn.Identity()
        # Initialize weights and apply final processing
        self.post_init()

    def forward(self, pixel_values,labels=None):
        # outputs = self.efficientnet(pixel_values).last_hidden_state[:, 0, :]  # CLS token representation
        # outputs = outputs.reshape(outputs.shape[0],-1)
        if self.training:
            outputs,aux=self.model(pixel_values)
            logits = [classifier(outputs) for classifier in self.classifiers]
            aux_logits = [classifier(aux) for classifier in self.classifiers2]
        else:
            outputs=self.model(pixel_values)
            logits = [classifier(outputs) for classifier in self.classifiers]
        
        if labels is not None:
            loss=0
            # loss_weights=np.array([1,1,1,1,1,1,1,5])
            # loss_weights=loss_weights/loss_weights.sum()
            for i in range(len(logits)):
                target=labels[:,i]
                loss += torch.nn.functional.cross_entropy(logits[i], target,ignore_index=-100)
                if self.training:
                    loss +=0.4*torch.nn.functional.cross_entropy(aux_logits[i], target,ignore_index=-100)
            return {"loss": loss, "logits": logits}
        return {"logits": logits}

    
# Example usage
num_labels = len(trackNum)  # For example, 5 different labels



In [25]:
config=CustomConfig(num_classes_per_label=trackNum)
model = MultiLabelMultiClassViT.from_pretrained(model_name, config=config,pretrain=True)
# # model.model.aux_logits=False

# model=MultiLabelMultiClassViT.from_pretrained(model_name)
print(summary(model))

Some weights of MultiLabelMultiClassViT were not initialized from the model checkpoint at Shadowking912/inception_v3 and are newly initialized: ['classifiers.0.bias', 'classifiers.0.weight', 'classifiers.1.bias', 'classifiers.1.weight', 'classifiers.2.bias', 'classifiers.2.weight', 'classifiers.3.bias', 'classifiers.3.weight', 'classifiers.4.bias', 'classifiers.4.weight', 'classifiers.5.bias', 'classifiers.5.weight', 'classifiers.6.bias', 'classifiers.6.weight', 'classifiers.7.bias', 'classifiers.7.weight', 'classifiers.8.bias', 'classifiers.8.weight', 'classifiers.9.bias', 'classifiers.9.weight', 'classifiers2.0.bias', 'classifiers2.0.weight', 'classifiers2.1.bias', 'classifiers2.1.weight', 'classifiers2.2.bias', 'classifiers2.2.weight', 'classifiers2.3.bias', 'classifiers2.3.weight', 'classifiers2.4.bias', 'classifiers2.4.weight', 'classifiers2.5.bias', 'classifiers2.5.weight', 'classifiers2.6.bias', 'classifiers2.6.weight', 'classifiers2.7.bias', 'classifiers2.7.weight', 'classifier

Layer (type:depth-idx)                   Param #
MultiLabelMultiClassViT                  --
├─Inception3: 1-1                        --
│    └─BasicConv2d: 2-1                  --
│    │    └─Conv2d: 3-1                  864
│    │    └─BatchNorm2d: 3-2             64
│    └─BasicConv2d: 2-2                  --
│    │    └─Conv2d: 3-3                  9,216
│    │    └─BatchNorm2d: 3-4             64
│    └─BasicConv2d: 2-3                  --
│    │    └─Conv2d: 3-5                  18,432
│    │    └─BatchNorm2d: 3-6             128
│    └─MaxPool2d: 2-4                    --
│    └─BasicConv2d: 2-5                  --
│    │    └─Conv2d: 3-7                  5,120
│    │    └─BatchNorm2d: 3-8             160
│    └─BasicConv2d: 2-6                  --
│    │    └─Conv2d: 3-9                  138,240
│    │    └─BatchNorm2d: 3-10            384
│    └─MaxPool2d: 2-7                    --
│    └─InceptionA: 2-8                   --
│    │    └─BasicConv2d: 3-11            12,416
│   

In [26]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import classification_report,f1_score
batch_size = 32
def collate_fn(batch):
    return {
        'pixel_values': torch.cat([x['pixel_values'] for x in batch],dim=0),
        'labels': torch.stack([x['labels'] for x in batch])
    }

def compute_metrics(pred):
    logits = pred.predictions
    labels=pred.label_ids
    probs = np.stack([np.argmax(logit,axis=1) for logit in logits])
    probs=probs.T
    f1s=[]
    for i in range(labels.shape[1]):
        non_padding_indices = [j for j, label in enumerate(labels[:,i]) if label != -100]
        labels_ = [labels[j,i] for j in non_padding_indices]
        probs_ = [probs[j,i] for j in non_padding_indices]
        micro=f1_score(labels_,probs_,average='micro')
        macro=f1_score(labels_,probs_,average='macro')
        print(f"attr_{i} f1 score: {macro}")
        # print(classification_report(labels_,probs_))
        score=2*(micro*macro)/(micro+macro)
        f1s.append(score)
    print()
    print()
    return {'score': sum(f1s)/len(f1s)}


    # logits = pred.predictions
    # labels=pred.label_ids
    # probs = np.stack([np.argmax(logit,axis=1) for logit in logits])
    # probs=probs.T
    # labels=labels.flatten()
    # probs=probs.flatten()
    # non_padding_indices = [i for i, label in enumerate(labels) if label != -100]
    # labels = [labels[i] for i in non_padding_indices]
    # probs = [probs[i] for i in non_padding_indices]
    # print(classification_report(labels,probs))
    # report=classification_report(labels,probs,output_dict=True)
    # return {'accuracy': report['accuracy'],"macro avg f1":report['macro avg']['f1-score']}

training_args = TrainingArguments(
  output_dir="./vit6/"+category,
  per_device_train_batch_size=128,
  per_device_eval_batch_size=128,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  logging_strategy="epoch",
  num_train_epochs=10,
  bf16=True,
  learning_rate=2e-4,
#   dataloader_num_workers=20,lr
  save_total_limit=1,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='wandb',
  load_best_model_at_end=True,
  metric_for_best_model="score"
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_fashion_data,
    eval_dataset=val_fashion_data,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    # tokenizer=processor,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer.train()
trainer.save_model(f"./vit6/{category}/final")

[2024-11-16 23:41:14,468] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/aseems/anaconda3/envs/mhcp4/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karan21258 (karan912). Use `wandb login --relogin` to force relogin


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Epoch,Training Loss,Validation Loss,Score
1,0.000000,nan,0.230480
2,0.000000,nan,0.230480
3,0.000000,nan,0.230480


attr_0 f1 score: 0.02695789059425423
attr_1 f1 score: 0.13382193268186754
attr_2 f1 score: 0.3011211959423385
attr_3 f1 score: 0.0343082114735658
attr_4 f1 score: 0.4962106102911847
attr_5 f1 score: 0.03840682788051209
attr_6 f1 score: 0.1236788136930115
attr_7 f1 score: 0.17709313264346188
attr_8 f1 score: 0.1879661859771258
attr_9 f1 score: 0.11366991437850604


attr_0 f1 score: 0.02695789059425423
attr_1 f1 score: 0.13382193268186754
attr_2 f1 score: 0.3011211959423385
attr_3 f1 score: 0.0343082114735658
attr_4 f1 score: 0.4962106102911847
attr_5 f1 score: 0.03840682788051209
attr_6 f1 score: 0.1236788136930115
attr_7 f1 score: 0.17709313264346188
attr_8 f1 score: 0.1879661859771258
attr_9 f1 score: 0.11366991437850604


attr_0 f1 score: 0.02695789059425423
attr_1 f1 score: 0.13382193268186754
attr_2 f1 score: 0.3011211959423385
attr_3 f1 score: 0.0343082114735658
attr_4 f1 score: 0.4962106102911847
attr_5 f1 score: 0.03840682788051209
attr_6 f1 score: 0.1236788136930115
attr_7 f1 s

In [ ]:
trainer.evaluate(test_fashion_data)
#1-54(imbalance),2-65,3-85,5-56,6-70,7-52(imbalance),8-51(imbalance),9-60(imbalance),10-49(imbalance)
#4-30,

KeyboardInterrupt: 